In [1]:
# install the eartgeneing-api and geemap
!pip install earthengine-api 
!pip install geemap

  Using cached earthengine_api-1.7.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached google_api_python_client-2.187.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached google_auth_httplib2-0.2.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached httplib2-0.31.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.2.0-py3-none-any.whl.metadata (2.6 kB)
Using cached earthengine_api-1.7.1-py3-none-any.whl (468 kB)
Using cached httplib2-0.31.0-py3-none-any.whl (91 kB)
Using cached google_api_python_client-2.187.0-py3-none-any.whl (14.6 MB)
Using cached google_auth_httplib2-0.2.1-py3-none-any.whl (9.5 kB)
Using cached uritemplate-4.2.0-py3-none-any.whl (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [earthengine-api] [earthengine-api]n-client]
  Using cached geemap-0.36.6-py3-none-any.whl.metadata (14 kB)
  Using cached eerepr-0.1.2-py3-none-any.whl.metadata (4.2 kB)
  Using cached geocoder-1.38.1-py2.py3-none-any.whl.metadata (14 kB)
  Using cached ipyevents-2.0.4-py3-n

In [2]:
import ee
# Trigger the authentication flow.
ee.Authenticate(force=True)

# Initialize the library.
# ee.Initialize(project='rock-appliance-477517-u5')

Enter verification code:  4/1Ab32j91cG6PUFM9V_IUhT2ygnGZcnDlkN9IfZB23HlHF-5eN5C6VTg30gbY



Successfully saved authorization token.


In [ ]:
import rasterio
import geemap

In [3]:
# initialize the project library
ee.Initialize(project='columbia-phd')

Define the administrative region and the tree cover paramaters used to define the forest base layer. In this case, we are choosing a minimum of 30% canopy cover, and at least 0.5 ha of contiugious forest (around 6 pixels). These definitions are consistent with the recommendations by Hansen et al, 2024 and Global Forest Watch.

In [5]:
# define administrative aoi
gaul = ee.FeatureCollection("FAO/GAUL/2015/level1") # get the FAO feature collection
peru = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Peru')) # Filter Peru first (ADM0_NAME = country)
aoi = peru.filter(ee.Filter.eq('ADM1_NAME', 'Madre de Dios')) # Filter Madre de Dios region (ADM1_NAME = admin1 region)

# set the tree cover extent parameters (we consider forest all that is at least 30% canopy cover, at least 0.5 ha continuous, this is consistent with Hansen data handling)
canopy_cover = ee.Number(30) # canopy cover threshold (%)
min_extent = ee.Number(6) # min size threshold (6 pixels ≈ 0.5 ha)

Load the Hansen 2024, Forest Change Dataset. This dataset contains tree cover extent at 2000 as well as annual tree cover loss all the way to 2024. 

In [6]:
# load the Hansen Global Forest Change data layer
gfc2024 = ee.Image("UMD/hansen/global_forest_change_2024_v1_12")

# get tree cover extent in 2000
canopyCover_2000 = gfc2024.select("treecover2000")

# mask the tree cover according to our tree cover parameters defined above
canopyCover30 = canopyCover_2000.gte(canopy_cover).selfMask() # all pixels in 2000 with at least 30% canopy cover, as defined by Hansen et al
contArea = canopyCover30.connectedPixelCount()
extent2000 = contArea.gte(min_extent).selfMask() # all pixels in 2000 with at least 0.5 ha of continuous forest

# Get projection and scale for visualization later on
prj = gfc2024.projection()
scale = prj.nominalScale()

From the forest cover extent at 2000, find all the areas that have not experienced any loss from 2001 to 2023.

In [7]:
# from the GFC dataset, get tree loss
treeLoss = gfc2024.select("loss")

# In the GFC dataset, any value in "loss" that is greater 0 idnicates a loss event. The value to this loss event indicates the year. We want no loss events, loss=0 
noLoss = treeLoss.eq(0)

# mask the 2000 extent by the noLoss
current_extent = extent2000.updateMask(noLoss)

# clip this to the aoi defined above
mdd_extent = current_extent.clip(aoi)

In [10]:
# get the nominal scale of the TerraClim data, our largest dataset
TerraClim = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
first = TerraClim.first()
TC_scale = first.projection().nominalScale()

print(TC_scale.getInfo())

4638.312116386398


In [12]:
# rescale and reproject the image to the TC data scale
mdd_extent_TC = mdd_extent.reduceResolution(
    reducer=ee.Reducer.mode(),
    maxPixels=1024
).reproject(
    crs=mdd_extent.projection().crs,
    scale=4638.312116386398
)

# export the gee image to the python environment
out = "/home/jovyan/MLEAEEE4000-DroughtAmazon2/mdd_extent_TCscale.tif"

geemap.ee_export_image(
    mdd_extent,
    filename=out,
    scale=4638.312116386398, # assign the scale value of the TC data
    region=aoi.geometry(),
)

#Once you want to read in image again:
#with rasterio.open(out) as src:
    #arr = src.read(1)

Generating URL ...
Please wait ...
Data downloaded to /home/jovyan/MLEAEEE4000-DroughtAmazon2/mdd_extent_TCscale.tif
